In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from sparknlp.annotator import LemmatizerModel

In [7]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Sentiment Analysis with PySpark") \
    .getOrCreate()

# Sample data
data = [("I love this product",),
        ("This is terrible",),
        ("It's okay",)]


df = spark.read.option("header","true").json("../data/json/NVIDIA/")
df = df.select("published date","article")

# Create Spark DataFrame
# df = spark.createDataFrame(data, ["text"])

# Define a function to calculate sentiment score
def calculate_sentiment(text):
    # Define positive and negative words
    # positive_words = ["love", "great", "awesome", "excellent"]
    positive_words = [
    "Love", "Happy", "Joy", "Excited", "Wonderful", "Amazing", "Fantastic", "Great", "Excellent", "Awesome",
    "Delight", "Bliss", "Grateful", "Hopeful", "Peaceful", "Success", "Thrilled", "Enthusiastic", "Optimistic",
    "Admire", "Brilliant", "Cheerful", "Ecstatic", "Energetic", "Fascinating", "Glorious", "Harmony", "Inspirational",
    "Magical", "Miracle", "Radiant", "Sensational", "Triumphant", "Vibrant", "Wholesome", "Affection", "Blessing",
    "Caring", "Charming", "Courage", "Dazzling", "Dream", "Empower", "Freedom", "Glowing", "Healing", "Honor",
    "Incredible", "Jubilant", "Kindness", "Liberate", "Marvelous", "Noble", "Paradise", "Rejoice", "Spectacular",
    "Sunny", "Transform", "Uplift", "Wondrous", "Abundant", "Blissful", "Calm", "Divine", "Euphoric", "Genuine",
    "Heartfelt", "Inspire", "Majestic", "Nurture", "Overflowing", "Radiate", "Serene", "Sparkling", "Tranquil",
    "Unconditional", "Vitality", "Warmth", "Zest", "Adoration", "Bountiful", "Celebrate", "Compassion", "Elegant",
    "Faith", "Gratitude", "Hope", "Imagination", "Lively", "Miraculous", "Passion", "Renew", "Serenity", "Splendid",
    "Treasure", "Vibrant", "Wisdom"]

    negative_words = [
    "Hate", "Sad", "Angry", "Disappointed", "Terrible", "Horrible", "Awful", "Disgusting", "Annoyed", "Frustrated",
    "Depressed", "Gloomy", "Miserable", "Dreadful", "Unhappy", "Painful", "Sorrow", "Regret", "Agony", "Grief",
    "Melancholy", "Despair", "Hopeless", "Lonely", "Heartbroken", "Wretched", "Suffering", "Apathetic", "Desperate",
    "Furious", "Enraged", "Bitter", "Jealous", "Hostile", "Offensive", "Resentful", "Pessimistic", "Gloomy",
    "Nauseating", "Repulsive", "Disappointing", "Disgusted", "Disheartened", "Disillusioned", "Dismal", "Distressed",
    "Disturbed", "Grieving", "Lament", "Languish", "Moody", "Morose", "Nervous", "Pessimism", "Rage", "Rueful",
    "Shameful", "Somber", "Sorrowful", "Stressful", "Tearful", "Tragic", "Troubled", "Unpleasant", "Upset", "Vexed",
    "Weary", "Woeful", "Worst", "Worried", "Abominable", "Abysmal", "Anguish", "Anxious", "Appalled", "Atrocious",
    "Dejected", "Detest", "Dismayed", "Dread", "Fretful", "Grumpy", "Horrified", "Humiliated", "Inferior", "Insecure",
    "Irked", "Lousy", "Mournful", "Nasty", "Panicked", "Pathetic", "Perplexed", "Repugnant", "Shocked", "Stressed",
    "Tense", "Tiresome", "Unbearable", "Uncertain", "Unhappy", "Unpleasant", "Unsettled"
    ]


    
    # Count positive and negative words in the text
    num_positive = sum(1 for word in text.split() if word.lower() in positive_words)
    num_negative = sum(1 for word in text.split() if word.lower() in negative_words)
    
    # Calculate sentiment score (positive words - negative words)
    sentiment_score = (num_positive - num_negative) / max(num_positive + num_negative, 1)
    
    return sentiment_score

# Define a UDF to apply sentiment analysis to Spark DataFrame
sentiment_udf = udf(calculate_sentiment, FloatType())

# Apply sentiment analysis to DataFrame
df = df.withColumn("sentiment_score", sentiment_udf(df["article"]))

# Show results
df.show()


+--------------------+--------------------+---------------+
|      published date|             article|sentiment_score|
+--------------------+--------------------+---------------+
|Fri, 17 Dec 2021 ...|3.53 -13%\n\n3.54...|            0.0|
|Tue, 03 Jan 2017 ...|3.12 -12%\n\n3.13...|            0.0|
|Tue, 03 Jan 2017 ...|3.12 -12%\n\n3.13...|            0.0|
|Thu, 05 Sep 2019 ...|6.69 -8%\n\n6.72 ...|            0.0|
|Tue, 08 Jan 2019 ...|11.6 -7%\n\n11.8 ...|            0.0|
|Wed, 10 Apr 2024 ...|The best graphics...|            0.0|
|Wed, 19 Sep 2018 ...|TweakTown's Ratin...|            0.0|
|Wed, 19 Sep 2018 ...|TweakTown's Ratin...|            0.0|
|Fri, 02 Jun 2023 ...|This transcript w...|            0.0|
|Thu, 16 Nov 2023 ...|Every day, cloud-...|            0.0|
|Wed, 05 Apr 2023 ...|MLCommons today r...|            0.0|
|Wed, 29 Jun 2022 ...|MLCommons’ latest...|            0.0|
|Tue, 13 Jun 2017 ...|One area that is ...|            0.0|
|Fri, 05 Apr 2024 ...|We can happily re.

In [9]:
df_df = df.toPandas()

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:708)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:752)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:675)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:641)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:617)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:574)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:532)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


array([0.], dtype=float32)